In [45]:
#import sys
#!{sys.executable} -m pip install keras==2.4.0
import tensorflow
import keras
import wandb
import pandas as pd
import numpy as np
import statistics as st
from sklearn.utils import class_weight
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Activation, Conv1D, Flatten, MaxPooling1D
from keras.callbacks import EarlyStopping
from tensorflow.python.keras import regularizers

In [11]:
wandb.init()
config = wandb.config

# Set parameters
config.vocab_size = 1000
config.maxlen = 1000
config.batch_size = 32
config.embedding_dims = 10
config.filters = 16
config.kernel_size = 3
config.hidden_dims = 250
config.epochs = 10

In [25]:
df = pd.read_csv('processed_data/clean_data.csv')
df.dropna()
delete_row = df[df['Years'] == 0].index
df = df.drop(delete_row)
df['Years'].unique()

array([2018, 2017, 2019])

In [26]:
X_train, X_test, y_train, y_test = train_test_split(df['Tweets with no Stopwords'], df['Years'], test_size = 0.33,
                                                   random_state = 2)

In [27]:
tokenizer = text.Tokenizer(num_words = config.vocab_size)
tokenizer.fit_on_texts(X_train.astype(str))
X_train_tokenized = tokenizer.texts_to_matrix(X_train.astype(str))
X_test_tokenized = tokenizer.texts_to_matrix(X_test.astype(str))
X_train_seq = sequence.pad_sequences(X_train_tokenized, maxlen = config.maxlen)
X_test_seq = sequence.pad_sequences(X_test_tokenized, maxlen = config.maxlen)

In [28]:
# One-hot encoding
y_train_encoded = LabelBinarizer().fit_transform(y_train)
y_test_encoded = LabelBinarizer().fit_transform(y_test)

In [36]:
label_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train.ravel())
print(label_weights)
label_weights = {i:label_weights[i] for i in range(len(label_weights))} # Create dictionary
print(label_weights)

[1.21118492 0.5432297  2.99831792]
{0: 1.21118492304933, 1: 0.5432297008596592, 2: 2.9983179213948445}


/usr/local/Cellar/jupyterlab/3.0.9/libexec/lib/python3.9/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass classes=[2017 2018 2019], y=[2017 2018 2018 ... 2017 2017 2018] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [34]:
# Check shapes
print(X_train_seq.shape)
print(X_test_seq.shape)
print(y_train_encoded.shape)
print(y_test_encoded.shape)

(702308, 1000)
(345913, 1000)
(702308, 3)
(345913, 3)


In [49]:
callback = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 3)

def define_cnn_model(dropout_rate):
    model = Sequential()
    model.add(Embedding(config.vocab_size, config.embedding_dims, input_length = config.maxlen))
    model.add(Conv1D(config.filters, config.kernel_size, padding = 'valid', activation = 'relu'))
    model.add(MaxPooling1D())
    model.add(Conv1D(config.filters, config.kernel_size, padding = 'valid', activation = 'relu'))
    model.add(Flatten())
    model.add(Dense(config.hidden_dims, activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(3, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [47]:
# Define the drop out grid
dropout_grid = [0.1, 0.5, 0.9]
l1_grid = [2**-5, 2**-6, 2**-7, 2**-8]
l2_grid = [2**-5, 2**-6, 2**-7, 2**-8]
tot = len(dropout_grid) * len(l1_grid) * len(l2_grid)

# Variables for the best result
best_history = [] # place holder
best_ind = 0
best_acc = 0

# Loop through each combination
pos = 0
for ii in dropout_grid:
    for jj in l1_grid:
        for kk in l2_grid:
            pos = pos + 1
            print("Fitting the ", pos, "/", tot , " model")
            # define the model
            curr_model = define_cnn_model(ii, jj, kk)
            
            # train the model
            curr_history = curr_model.fit(X_train_seq, y_train_encoded, epochs = 5, class_weight = label_weights,
                                          batch_size = 64, validation_data = (X_test_seq, y_test_encoded), 
                                          callbacks = [callback], verbose = 1)
            curr_acc = st.mean(curr_history.history['val_acc'])
            
            # save the best result
            if best_acc < curr_acc:
                best_acc = curr_acc
                best_ind = pos - 1
                best_history = curr_history
                
print(best_acc)
print(best_ind)

Fitting the  1 / 48  model
Train on 702308 samples, validate on 345913 samples
Epoch 1/5
702308/702308 [==============================] - 953s 1ms/step - loss: 0.6446 - acc: 0.6431 - val_loss: 0.6373 - val_acc: 0.6860
Epoch 2/5
702308/702308 [==============================] - 934s 1ms/step - loss: 0.5710 - acc: 0.6736 - val_loss: 0.6354 - val_acc: 0.6782
Epoch 3/5
702308/702308 [==============================] - 927s 1ms/step - loss: 0.5486 - acc: 0.6952 - val_loss: 0.6211 - val_acc: 0.6963
Epoch 4/5
702308/702308 [==============================] - 967s 1ms/step - loss: 0.5290 - acc: 0.7123 - val_loss: 0.6096 - val_acc: 0.7033
Epoch 5/5
702308/702308 [==============================] - 1119s 2ms/step - loss: 0.5099 - acc: 0.7269 - val_loss: 0.6068 - val_acc: 0.7145
Fitting the  2 / 48  model
Train on 702308 samples, validate on 345913 samples
Epoch 1/5
702308/702308 [==============================] - 990s 1ms/step - loss: 0.6261 - acc: 0.6496 - val_loss: 0.6030 - val_acc: 0.7151
Epoch 2

702308/702308 [==============================] - 859s 1ms/step - loss: 0.5370 - acc: 0.7050 - val_loss: 0.6028 - val_acc: 0.7027
Epoch 4/5
702308/702308 [==============================] - 857s 1ms/step - loss: 0.5150 - acc: 0.7231 - val_loss: 0.6120 - val_acc: 0.7057
Epoch 5/5
702308/702308 [==============================] - 854s 1ms/step - loss: 0.4947 - acc: 0.7373 - val_loss: 0.5940 - val_acc: 0.7076
Fitting the  12 / 48  model
Train on 702308 samples, validate on 345913 samples
Epoch 1/5
702308/702308 [==============================] - 922s 1ms/step - loss: 0.6037 - acc: 0.6556 - val_loss: 0.6421 - val_acc: 0.6373
Epoch 2/5
702308/702308 [==============================] - 920s 1ms/step - loss: 0.5598 - acc: 0.6834 - val_loss: 0.6442 - val_acc: 0.6377
Epoch 3/5
702308/702308 [==============================] - 956s 1ms/step - loss: 0.5361 - acc: 0.7061 - val_loss: 0.5793 - val_acc: 0.7350
Epoch 4/5
702308/702308 [==============================] - 952s 1ms/step - loss: 0.5130 - acc: 0

702308/702308 [==============================] - 854s 1ms/step - loss: 0.6215 - acc: 0.6468 - val_loss: 0.6319 - val_acc: 0.6819
Epoch 2/5
702308/702308 [==============================] - 866s 1ms/step - loss: 0.5789 - acc: 0.6657 - val_loss: 0.6232 - val_acc: 0.6869
Epoch 3/5
702308/702308 [==============================] - 857s 1ms/step - loss: 0.5691 - acc: 0.6799 - val_loss: 0.6332 - val_acc: 0.6669
Epoch 4/5
702308/702308 [==============================] - 865s 1ms/step - loss: 0.5607 - acc: 0.6949 - val_loss: 0.6344 - val_acc: 0.6738
Epoch 5/5
702308/702308 [==============================] - 861s 1ms/step - loss: 0.5540 - acc: 0.7012 - val_loss: 0.6170 - val_acc: 0.6798
Fitting the  23 / 48  model
Train on 702308 samples, validate on 345913 samples
Epoch 1/5
702308/702308 [==============================] - 857s 1ms/step - loss: 0.6372 - acc: 0.6360 - val_loss: 0.7083 - val_acc: 0.5855
Epoch 2/5
702308/702308 [==============================] - 861s 1ms/step - loss: 0.6029 - acc: 0

Epoch 4/5
702308/702308 [==============================] - 959s 1ms/step - loss: 0.5561 - acc: 0.6965 - val_loss: 0.6249 - val_acc: 0.6800
Epoch 5/5
702308/702308 [==============================] - 1046s 1ms/step - loss: 0.5493 - acc: 0.7047 - val_loss: 0.6288 - val_acc: 0.6743
Fitting the  33 / 48  model
Train on 702308 samples, validate on 345913 samples
Epoch 1/5
702308/702308 [==============================] - 1002s 1ms/step - loss: 0.6995 - acc: 0.5929 - val_loss: 0.6702 - val_acc: 0.6163
Epoch 2/5
702308/702308 [==============================] - 1013s 1ms/step - loss: 0.6486 - acc: 0.6095 - val_loss: 0.6591 - val_acc: 0.6351
Epoch 3/5
702308/702308 [==============================] - 1001s 1ms/step - loss: 0.6310 - acc: 0.6318 - val_loss: 0.6580 - val_acc: 0.6267
Epoch 4/5
702308/702308 [==============================] - 999s 1ms/step - loss: 0.6255 - acc: 0.6463 - val_loss: 0.6519 - val_acc: 0.6340
Epoch 5/5
702308/702308 [==============================] - 996s 1ms/step - loss: 0

702308/702308 [==============================] - 877s 1ms/step - loss: 0.6271 - acc: 0.6343 - val_loss: 0.6584 - val_acc: 0.6276
Epoch 3/5
702308/702308 [==============================] - 865s 1ms/step - loss: 0.6242 - acc: 0.6415 - val_loss: 0.6553 - val_acc: 0.6312
Epoch 4/5
702308/702308 [==============================] - 866s 1ms/step - loss: 0.6221 - acc: 0.6492 - val_loss: 0.6533 - val_acc: 0.6358
Epoch 5/5
702308/702308 [==============================] - 873s 1ms/step - loss: 0.6201 - acc: 0.6534 - val_loss: 0.6512 - val_acc: 0.6337
Fitting the  44 / 48  model
Train on 702308 samples, validate on 345913 samples
Epoch 1/5
702308/702308 [==============================] - 874s 1ms/step - loss: 0.6781 - acc: 0.5837 - val_loss: 0.6647 - val_acc: 0.6210
Epoch 2/5
702308/702308 [==============================] - 871s 1ms/step - loss: 0.6354 - acc: 0.6127 - val_loss: 0.6247 - val_acc: 0.6814
Epoch 3/5
702308/702308 [==============================] - 864s 1ms/step - loss: 0.6270 - acc: 0

In [ ]:
model = define_cnn_model(0.5)
model.summary()
history = model.fit(X_train_seq, y_train_encoded, epochs = 15, class_weight = label_weights, batch_size = 64, 
                    validation_data = (X_test_seq, y_test_encoded), verbose = 1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_53 (Embedding)     (None, 1000, 10)          10000     
_________________________________________________________________
conv1d_105 (Conv1D)          (None, 998, 16)           496       
_________________________________________________________________
max_pooling1d_53 (MaxPooling (None, 499, 16)           0         
_________________________________________________________________
conv1d_106 (Conv1D)          (None, 497, 16)           784       
_________________________________________________________________
flatten_53 (Flatten)         (None, 7952)              0         
_________________________________________________________________
dense_105 (Dense)            (None, 250)               1988250   
_________________________________________________________________
dropout_56 (Dropout)         (None, 250)               0         
__________